In [1]:
import numpy as np
import json
import os
import csv

In [2]:
#creating training & test data file: 18*2*45*n => 30 frame overlap

In [12]:
asanas = ["bhujangasan", "padamasan", "shavasan", "tadasan", "trikonasan", "vrikshasan"]
data_path = "pose-data/"

In [15]:
train_list = []
#list of asanas, each asana has all it's videos, each video has list of stacked 18*2 numpy arrays
val_list = []
test_list = []
for a in asanas:
    currAsanaTrain = []
    currAsanaVal = []
    currAsanaTest = []
    path = data_path + a + "/"
    for i in range(1,17):
        currVideo = []
        start = str(i) + "_"
        for filename in os.listdir(path):
            data = []
            if filename.startswith(start):
                #get data from file
                with open(path + filename) as json_data:
                    d = json.load(json_data)
                    try:
                        data = d['people'][0]['pose_keypoints']
                        #remove confidence values
                        for j in range(2,38,2):
                            data.pop(j)
                    except:
                        print ("Failed at" + filename)
                        continue
                    json_data.close
                #separate X and Y coord
                npdata = np.asarray(data)
                Xdata = data[::2]
                Ydata = data[1::2]
                stk = np.dstack((Xdata, Ydata)) #stack vertically
                currVideo.append(stk)
        #print currVideo
        if a == 'vrikshasan' and i == 15:
            # this one has difference and creates noise
            currAsanaTrain.append(currVideo)
        elif (i+1)%5 == 0 and len(currVideo) != 0:
            currAsanaTest.append(currVideo)
        elif (i)%5 == 0 and len(currVideo) != 0:
            currAsanaVal.append(currVideo)
        elif len(currVideo) != 0:
            currAsanaTrain.append(currVideo)
        #break
    train_list.append(currAsanaTrain)
    val_list.append(currAsanaVal)
    test_list.append(currAsanaTest)
    #break

In [16]:
#videos in datset are 30 fps
window_size = 45 # number of frames in 1 test case
overlap_size = 36 # overlap between two testcase
train_cases = []
train_labels = []
for i, asana in enumerate(train_list):
    for vid in asana:
        for start in range(0, len(vid)-window_size, window_size - overlap_size):
            currCase = np.empty([window_size,18,2])
            for index in range(0,window_size):
                currCase[index] = vid[start+index]
            train_cases.append(currCase)
            train_labels.append(i)

In [17]:
print (len(train_cases))
print (len(train_labels))

0
0


In [18]:
test_cases = []
test_labels = []
for i, asana in enumerate(test_list):
    for vid in asana:
        for start in range(0, len(vid)-window_size, window_size - overlap_size):
            currCase = np.empty([window_size,18,2])
            for index in range(0,window_size):
                currCase[index] = vid[start+index]
            test_cases.append(currCase)
            test_labels.append(i)

In [19]:
print (len(test_cases))
print (len(test_labels))

0
0


In [20]:
val_cases = []
val_labels = []
for i, asana in enumerate(val_list):
    for vid in asana:
        for start in range(0, len(vid)-window_size, window_size - overlap_size):
            currCase = np.empty([window_size,18,2])
            for index in range(0,window_size):
                currCase[index] = vid[start+index]
            val_cases.append(currCase)
            val_labels.append(i)

In [21]:
print (len(val_cases))
print (len(val_labels))

0
0


In [22]:
#convert to numpy array and save
train_arr = np.empty([len(train_cases), window_size, 18, 2])
for i, ele in enumerate(train_cases):
    train_arr[i] = ele

In [23]:
test_arr = np.empty([len(test_cases), window_size, 18, 2])
for i, ele in enumerate(test_cases):
    test_arr[i] = ele

In [24]:
val_arr = np.empty([len(val_cases), window_size, 18, 2])
for i, ele in enumerate(val_cases):
    val_arr[i] = ele

In [25]:
np.save("trainX", train_arr)
np.save("testX", test_arr)
np.save("valX", val_arr)
np.save("trainY", np.asarray(train_labels))
np.save("testY", np.asarray(test_labels))
np.save("valY", np.asarray(val_labels))